## Procedimentos Globais

In [ ]:
global_dict_sutdentes_profiles = {}


## Conexão com o banco

Primeiramente precisamos importar algumas configurações e nos conectar a nossa base de dados


In [ ]:
# Caso utilize o Google Colab, descomente as linhas abaixo para montar o Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Importar bibliotecas necessárias
from urllib.parse import quote_plus
import socket
import os

# Importar pymongo para acesso aos documentos no banco de dados
import pymongo

# Importar configparser e pprint para ler o arquivo de configuração e imprimir os dados
import configparser
import pprint

hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)
print("Nome do seu computador:" + hostname)
print("O endereço IP do seu computador:" + IPAddr)

config = configparser.ConfigParser()
config.read('./conf.ini')

# Configurações do MongoDB
username = config['mongo']['USERNAME']
password = config['mongo']['PASSWORD']
host = config['mongo']['HOST']


# Criar string de conexão com o banco
connection_string = 'mongodb://{}:{}@{}/?retryWrites=true&w=majority'.format(
        username,
        quote_plus(password),
        host
    )

# Conectar ao MongoDB
client = pymongo.MongoClient(connection_string)

## Tratamento de dados




### Passo 1: realizar busca das informações para um aluno
O resultado é emitido para um determinado aluno, para isso é necessário os resultados dos formulários referentes as suas turmas durante o ensino médio (1°,2° e 3° ano).



In [ ]:

from pipelines import *

db_connection = client[config['mongo']['DB_NAME']]
formulario_collection = db_connection['formularios_professor']
escola_collection = db_connection['escolas']
competencia_collection = db_connection['competencias']


CHOSEN_SCHOOL = config['params']['school_name']
CHOSEN_STUDENT = config['params']['student_name']

escola = escola_collection.find_one({'nome': CHOSEN_SCHOOL})

turmas_do_aluno = escola_collection.aggregate(get_turmas_do_aluno_pipeline(CHOSEN_SCHOOL, CHOSEN_STUDENT))

print(turmas_do_aluno)

lista_de_turmas_do_aluno = (list(turmas_do_aluno))

print("Turmas do aluno {} na escola {}:".format(CHOSEN_STUDENT, CHOSEN_SCHOOL))

pprint.pprint(lista_de_turmas_do_aluno)


### Passo 2: Retornar Formulários que pertencem as turmas do aluno escolhido




Para que seja possível construir o perfil é necesssário pelo menos um formulários preecnhidos para cada ano letivo, ou seja os formulários do 1°,2° e 3° ano. Utilizando a pipeline do mongo db retornaremos quantos formulários foram preenchidos para cada ano, por exemplo

```python
  numDisplAval: {
    ano3: 1,
    ano1: 6,
    ano2: 3
}
```

Sendo assim utilizamos o estágio na pipeline



```python
{
    '$set': {
            'numDisplAval.ano1': {
                '$size': {
                    '$filter': {
                        'input': '$seriesAvaliadas',
                        'as': 'nota',
                        'cond': {
                            '$eq': [
                                '$$nota', 1
                            ]
                        }
                    }
                }
            },
            'numDisplAval.ano2': {
                '$size': {
                    '$filter': {
                        'input': '$seriesAvaliadas',
                        'as': 'nota',
                        'cond': {
                            '$eq': [
                                '$$nota', 2
                            ]
                        }
                    }
                }
            },
            'numDisplAval.ano3': {
                '$size': {
                    '$filter': {
                        'input': '$seriesAvaliadas',
                        'as': 'nota',
                        'cond': {
                            '$eq': [
                                '$$nota', 3
                            ]
                        }
                    }
                }
            }
        }
    }
```



### Passo 3: cálculo do somáorio da competência em todas as disciplinas



Antes de mais nada, para ambos os perfils contruídos precisamos do somatório das competências referentes a todas as disciplinas achar o peso $k_{ij}$.

\begin{equation}
  k_{ij} = \dfrac{\mathcal{C}_{i j} }{\sum^{N}_{i=0}\mathcal{C}_{ij}}
\end{equation}

A soma das competências por disciplina é necessário para suas normalizações, esse valor so é adiquirido uma vez que todas as disciplinas que a competência $j$ está relacionada são preechidas. Sendo $N$ o número total de disciplinas, variando de $i = \{ 0 \cdots N -1 \}$ obtemos:

\begin{equation}
    \sum^{N}_{i=0}\mathcal{C}_{ij}
\end{equation}

O valor do peso que relaciona a competência com a disciplina ainda está na forma do valor atribuído pelo professor no questionário, o que é necessário então executar a média das habilidades atribuídas.
Para cada habilidade $\mathcal{H}_{k}$ em um número de habilidades $k$, obtem-se o atribuído da competência.

\begin{equation}
    \mathcal{C}_{i j} = \dfrac{1}{L}\sum^{L}_{l=0}\mathcal{H}_{l}
\end{equation}

Postariormente o necessário é normalizar este valor em relação a soma dos valores da competências para todas as disciplinas. O que será visto no próximo passo.



```
# Média das Habilidades
{
    '$set': {
      'grafos.arestas.valor': {
        '$avg': '$grafos.arestas.valor',
      },
    },
}

# Somatório das competências
{
    '$addFields': {
      'sumValorCompetencias': {
        '$sum': '$arestas.valor',
      },
    },
},
```



### Passo 4: Retornar formulário

Utilizando a função de agregação do mongo para retornar os dados formatados, sendo os formulários pertencentes as turmas do aluno, os formulários contém quantas disciplinas de cada foram respondidas para a competência associada

In [ ]:

ids_turmas = [turma['_id'] for turma in lista_de_turmas_do_aluno]
print("IDs das turmas do aluno {}: {}".format(CHOSEN_STUDENT, ids_turmas))
# Obter as competências avaliadas nas turmas do aluno

competencias_avaliadas = list(formulario_collection.aggregate(get_competencias_avaliadas_pipeline(ids_turmas)))

#pprint.pprint(competencias_avaliadas)


### Passo 5: checar formulários inválidos

Como foi dito no dado anterior temos as disciplinas respondidas para cada ano . Avalie se o numero de arestas é igual ao numero de elementos das matrizes

In [ ]:

total_competencia_por_area = list(competencia_collection.aggregate(get_competencia_por_area_pipeline()))
print("Total de competências por área:")
pprint.pprint(total_competencia_por_area)

[result] = list(escola_collection.aggregate(get_disciplina_por_area_pipeline(CHOSEN_SCHOOL)))
numDiscip = result["numDiscip"]  # Acessa o valor pela chave
print("Número de disciplinas por área:")
pprint.pprint(numDiscip)

[ result ] = list(formulario_collection.aggregate(get_arestas_total_pipeline(ids_turmas)))
totalArestaGeral = result['totalArestasGeral']
print("Total de arestas gerais:")
pprint.pprint(totalArestaGeral)
# Verificar se os formulários estão válidos
acumulador = 0
for comp in total_competencia_por_area:
  if comp['tipo'] == 'COGNITIVOS':
    acumulador += comp['total'] * sum(numDiscip.values())
  else:
    acumulador += comp['total']* numDiscip[comp['tipo']]
print("Total de arestas acumuladas:")
pprint.pprint(acumulador)

invalidos =  totalArestaGeral != acumulador

if (invalidos):
  print('Formulários inválidos, resultados não serão coerentes!')
  
else:
  print('Formulários válidos')


### Passo 6: Recupaerar valor notas do histórico normalizado
Vamos extrair o valor das notas do histórco para cada disciplina e utilizar em um dicionário, associando o id da disciplina a nota obtida pelo aluno. O valor de uma nota $g_{dp}$ será $G_{dp}$ (ou $S(u)$)após dividirmos pelo valor maximo que uma nota pode alcançar ($g_{max}$).

\begin{equation}
G_{dp} = \frac{g_{dp}}{g_{max}}
\end{equation}

In [ ]:
NOTA_MAXIMA_ESCOLA = float(config['params']['nota_max_escola'])
nota_historico_dict ={}
print(lista_de_turmas_do_aluno)
for turma in lista_de_turmas_do_aluno:
  for disciplina in turma['aluno']['notas']['historico_escolar']: #para cada disciplina d
    print("Disciplina: {}".format(disciplina['disciplina_id']))
    nota_historico_dict[(disciplina['disciplina_id'])] = disciplina['nota']/NOTA_MAXIMA_ESCOLA # teremos um valor G_dp

#pprint.pprint(nota_historico_dict)


### Passo 7: Recuperar valor de notas do ENEM

Na nova estrutura para cada disciplina, encontrar a disciplina respectiva do aluno, primeiramente mapearemos as disciplinas e suas notas. Assim como no histórico dividiremos pelo máximo do ENEM.

\begin{equation}
\mathcal{C}_A = \frac{En_{A}}{N_{cA}}
\end{equation}

Com uma média desses valores utilizaremos o perfil para eixos cognitivos

\begin{equation}
\mathcal{C}_C = \frac{1}{4} \cdot \sum^{4}_{A=0}\mathcal{C}_{A}
\end{equation}

In [ ]:
#fazer pequeno filtro para nota do enem
list_notas_enem = [turma['aluno']['notas']['enem'] for turma in lista_de_turmas_do_aluno]
nota_enem = list_notas_enem[2].copy()

N_ca = 1000.0 #dividir todos os valores por 1000

Acc= 0.0

for key, value in nota_enem.items():
  nota_enem[key] = float("%.3f"%(value/N_ca))
  Acc += nota_enem[key]
nota_enem['MEDIA'] = float("%.3f"%(Acc/4))



pprint.pprint(nota_enem)

### Passo 8: Recuperar valores de aresta normalizado e de notas
Incluindo todas as disciplinas dos conseguimos obter o valor normalizado das arestas, também aproveitamos para remodelar a estrutura de cada competência e seu respectivo grafo. Portanto precisamos obter o valor $W(u,v)$ sendo $u$ uma disciplina que se conecta as próximas do próximo ano $v$. O valor da conexão $W(u,v)$ será equivalente ao valor da competência normalizado. Trazendo para o contexto de variáveis exestirá uma aresta que tem o valor da competência, a aresta terá uma origem em uma displina ($u$) mas seu nome se repete então $u$ terá um nome $d$ e um período $p$ assim o valor da competência referente a eles se torna o $C_{dp}$

\begin{equation}
    k_{i j} = \dfrac{\mathcal{C}_{ij}}{\sum^{N}_{i=0}\mathcal{C}_{ij}}
\end{equation}

\begin{equation}
    k_{i j} = W(u,v)
\end{equation}

\begin{equation}
    C_{dp} = W(u,v)
\end{equation}

Também ja podemos obter a nota em relação ao Ensino Médio $S(u)$ ou $G_{dp}$, assim as competencias contem o grafo, cada vertice de origem $u$ e seus destinos $v$ representados por objetos de disciplina

In [ ]:

competencia_grafos = {}
competencia_tag = {}

for competencia in competencias_avaliadas:
  #print("Competência: {}".format(competencia['_id']))
  competencia_grafos[competencia['_id']] = []
  competencia_tag[competencia['_id']] = competencia['competencia_tag']
  #print(competencia['arestas'])
  for aresta in competencia['arestas']:
    if(aresta['origem']['serie_ano'] < 4):
      competencia_grafos[competencia['_id']].append({
          'u':aresta['origem'],
          'v':aresta['destino'],
          'C_dp':aresta['valor']/competencia['sumValorCompetencias'],
          'G_dp':nota_historico_dict[str(aresta['origem']['_id'])]
      })

competencia_grafos = dict(sorted(competencia_grafos.items()))

#pprint.pprint(competencia_grafos)


### Passo 9: Contruir Grafo Para Vizualização.

Para que o grafo seja melhor vizualizado e analisado, vamos utilizar libs que proporcionam isso. Vamos construir uma função para utilizar essa estrutura, outra para impressão caso desejamos. Vamos construir uma função para utilizar essa estrutura, outra para impressão caso desejamos.

### Passo 10: Construir o perfil de competências do EM 

O valor referente a saída de um nó se da por:

\begin{equation}
    y_{dp}= G_{dp} + \sum^{T_{p-1}}  Y_{p-1}
\end{equation}

Sendo um  $Y_{dp}$ como a saída de um nó multiplicado pelo peso da aresta

\begin{equation}
    Y_{dp}=C_{dp} * y_{dp}  
\end{equation}

Assumimos que nas condições iniciais $Y_{d1} = G_{d1} * C_{d1}$ e na saída do nó final $Y_{d3} = y_{d3}$, então tentaremos criar um algoritmo para cada grafo que faça um acúmulo dos pesos e propaue o valor para frente.



### Passo 11: Testar

Precisamos testar agora, escolha qualquer competência, talvez uma diferente do exemplo, execute a vizualização do grafo, faça os cáculos com a função de propagação e veja se os valores procedem

In [ ]:
from plot_graphs import plot_graph
from graph_competencias import GraphCompetences

# Selecione a competência desejada
test_competence = config['params']['chosen_competence_area_id']
# Caso queira testar com outra competência, altere o valor de chosen_competence_area_id.Se for uma compet^ncia dos eixos cognivos descomente a linha abaixo:
# test_competence = config['params']['chosen_competence_cog_id']

NOTA_MINIMA = float(config['params']['nota_min_escola'])
razao_minima = NOTA_MAXIMA_ESCOLA/ NOTA_MINIMA

# Criar o grafo e plotar
test_G = GraphCompetences(competencia_grafos[test_competence],razao_minima=razao_minima)

# Plotar o grafo
plot_graph(test_G)
# Propagar valores no grafo

# Plotar o grafo com os valores propagados
print('Saída',test_G.output_competencia)

### Passo 12: Construir perfil de aluno máximo e aluno mínimo (normalização)

Para a normalização, teremos de fazer o perfil de um aluno máximo , com todas as notas máximas e um aluno mínimo, o mínimo requirido para passar. A forma do aluno máximo pode ser concretizada por:

\begin{equation}
    y_{fmax}=\sum_{k=1}^3\Big(\prod_{p=k}^ 3\mathcal{A}_p\Big)
\end{equation}

\begin{equation}
y_{fmax}=(1+\mathcal{A}_1)\mathcal{A}_2)\mathcal{A}_3
\end{equation}


\begin{equation}
\mathcal{A}_p = \sum_{k=0}^3C_{dp}
\end{equation}

Para o valor mínimo, usando o valor de nota míma para passar como 0,6:

\begin{equation}
y_{fmin} = 0,6 \cdot y_{fmax}
\end{equation}



In [ ]:
print("Aluno máximo", test_G.aluno_maximo)
print("Aluno mínimo", test_G.aluno_minimo)

### Passo 12: Construir os vetores de perfis.

Finalmente precisamos do valor normalizado de $y$ sendo $y'$
\begin{equation}
y'_{f}=\frac{y_f-y_{fmin}}{y_{fmax}-y_{fmin}}
\end{equation}

cada saída $\gamma_{j}$ representa a saída de uma competência da matrix assim teremos:

In [ ]:
perfil_enem = {}
perfil_em = {}

for key,value in competencia_grafos.items():
    tag = competencia_tag[key]
    perfil_enem[key] = nota_enem['MEDIA'] if tag == 'COGNITIVOS' else nota_enem[tag]
    grafo_competencia = GraphCompetences(value)
    
    competencia_do_aluno = grafo_competencia.output_competencia
    print("Competência:", tag,)
    print("Valor do aluno:", competencia_do_aluno)

    competencia_aluno_maximo = grafo_competencia.aluno_maximo
    competencia_aluno_minimo = grafo_competencia.aluno_minimo
    print("Valor Máximo do aluno:", competencia_aluno_maximo)
    print("Valor Mínimo do aluno:", competencia_aluno_minimo)

    perfil_em[key] = (competencia_do_aluno - competencia_aluno_minimo ) / (competencia_aluno_maximo - competencia_aluno_minimo)

    if (perfil_em[key] < 0):
        print("Perfil EM negativo!!")
    if (perfil_em[key] > 1):
        print("Perfil EM maior que 1!!")

# print("Perfil ENEM:", perfil_enem)
print("Perfil EM:", perfil_em)

# Salvar o perfil do aluno no dicionário global
global_dict_sutdentes_profiles[CHOSEN_STUDENT] = {
    'perfil_enem': perfil_enem,
    'perfil_em': perfil_em
}
# Salvar o dicionário global em um arquivo JSON
import json
output_file = os.path.join(os.getcwd(), 'student_profiles.json')
with open(output_file, 'w') as f:
    json.dump(global_dict_sutdentes_profiles, f, indent=4)
print(f"Perfil do aluno {CHOSEN_STUDENT} salvo em {output_file}")



## Exibir indicador de similaridade (plot de gráficos)


### Gráfico geral
Baseado nos estudos anteriores a comparação dos perfis seria de tal forma. Apresentaremos o grafico mediante a cada competência, como o perfil do ensino médio apenas utilizamos a nota, usamos uma constante, que deverá ser comparado as competências, elas terima que no caso se adequar aquela contante da nota

In [ ]:

from plot_graphs import bar_plot, spider_plot

competencias_dict = list(competencia_collection.aggregate(get_legendas_colunas_pipeline()))[0]

load_students_profiles = json.loads(open(output_file, 'r').read())
print("Perfis de alunos carregados:", load_students_profiles)

# print(competencias_dict)

cores = {
    'EXATAS': '#394fe3',
    'HUMANAS': '#e3dd39',
    'LINGUAGENS': '#d919e3',
    'MEDIA': '#6007b3',
    'COGNITIVOS': '#e31c19',
    'NATUREZA': '#39e347'
}

columns = [ competencias_dict[key]['codigo'] for key in perfil_em.keys() ]

columns_colors = [ cores[competencias_dict[key]['area']] for key in perfil_em.keys() ]
values_em = list(perfil_em.values())
values_enem = list(perfil_enem.values())

# print("Colunas:", columns)
# print("Cores:", columns_colors)


constant_lines = [
    {
        'name': 'Vestibular - {}'.format(key),
        'y': value,
        'color': cores[key],
        'dash': 'dot' if key == 'MEDIA' else 'dash'  # Linha da média com estilo diferente
    } 
    for key, value in nota_enem.items()
]

print("Perfil de", CHOSEN_STUDENT, 'na escola "', CHOSEN_SCHOOL, '"')

fig = bar_plot(
    columns,
    'Competências', 
    values_em,
    True,
    constant_lines,
    columns_colors
)
spider_fig = spider_plot(columns,['Egresso',' Vestibulando'], [values_em, values_enem], show=True)


A vizualização não fica tão agradável, portanto vamos seprar em graficos para cada área.

In [ ]:
#fazer um plot para cada area
area_values_dict = {}
for area in ['EXATAS', 'HUMANAS', 'LINGUAGENS', 'NATUREZA']:
    area_columns = [competencias_dict[key]['codigo'] for key in perfil_em.keys() if competencias_dict[key]['area'] == area or competencias_dict[key]['area'] == 'COGNITIVOS']
    area_values_em = [perfil_em[key] for key in perfil_em.keys() if competencias_dict[key]['area'] == area or competencias_dict[key]['area'] == 'COGNITIVOS']
    
    area_colors = [cores[competencias_dict[key]['area']] for key in perfil_em.keys() if competencias_dict[key]['area'] == area or competencias_dict[key]['area'] == 'COGNITIVOS']

    constant_lines = [
        {
            'name': 'Vestibular - {}'.format(area),
            'y': nota_enem[area],
            'color': cores[area],
            'dash': 'dash'  # Linha da média com estilo diferente
        },
        {
            'name': 'Média - total',
            'y': nota_enem['MEDIA'],
            'color': cores['MEDIA'],
            'dash': 'dot'  # Linha da média com estilo diferente
        } 
    ]
    print(f"Plotando área: {area}")
    print("Colunas:", area_columns)
    print("Cores:", area_colors)
    print("Valores EM:", area_values_em)

    fig_area = bar_plot(
        area_columns,
        f'Competências - {area}',
        area_values_em,
        True,
        constant_lines,
        area_colors
    )

    area_values_dict[area] = area_values_em

Agora utilizaremos a média das áreas pare montar um gráfico similar ao dos perfil do ensino médio. Usaremos a média entre as competências presentes na área.

In [ ]:
perfil_enem_values = [ nota_enem[key] for key in ['EXATAS', 'HUMANAS', 'LINGUAGENS', 'NATUREZA']]
perfil_media_area_values_em = [sum(area_values_dict[area]) / len(area_values_dict[area]) for area in ['EXATAS', 'HUMANAS', 'LINGUAGENS', 'NATUREZA']]

spider_fig = spider_plot( ['EXATAS', 'HUMANAS', 'LINGUAGENS', 'NATUREZA'],['Egresso',' Vestibulando'], [perfil_media_area_values_em, perfil_enem_values], show=True)


Por fim, buscando obter o  índice de similaridade entre
dois perfis de competencias, para que este represente um
valor numerico entre 0% e 100%. Utilizamos a similaridade por coseno e ajustamos para a proporcionalidade.

\begin{equation}
    \theta = cos^{-1} \Bigg( \dfrac{{P}_{em} \cdot {P}_{enem} }{||{P}_{em}|| \cdot ||{P}_{enem}||} \Bigg)
\end{equation}

Assumimos que a maior diferença entre os vetores é de $\pi/2$ rad, dessa forma ajustamos a escala para o valor máximo da diferença ser 1.

\begin{equation}
  Id = 1- \frac{2\theta}{\pi}
\end{equation}


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import math
import numpy as np

em_array = np.array(values_em)
enem_array = np.array(values_enem)

similarity_cos = cosine_similarity([em_array], [enem_array])[0][0]

theta = math.acos(similarity_cos)
similarity = 1-theta/(math.pi/2.0)

print('Similaridade de {}% entre os perfis do ensino médio e nem para as competências'.format(similarity * 100))


em_array = np.array(perfil_media_area_values_em)
enem_array = np.array(perfil_enem_values)

similarity_cos = cosine_similarity([em_array], [enem_array])[0][0]

theta = math.acos(similarity_cos)
similarity = 1-theta/(math.pi/2.0)

print('Similaridade de {}% entre os perfis da média'.format(similarity * 100))





In [ ]:
for key,profile in load_students_profiles.items():
    print("Perfil do aluno {}".format(key))
    perfil_em = profile['perfil_em']
    perfil_enem = profile['perfil_enem']
    columns = [ competencias_dict[key]['codigo'] for key in perfil_em.keys() ]
    bar_plot(columns, [ key + ' - Egresso', key + ' - Vestibulando'], [list(perfil_em.values()), list(perfil_enem.values())], True)

In [ ]:

#dos perfis no json quero o perfil aluno humanas e aluno natureza
chosens_em = ['Aluno Linguagens Escola A', 'Aluno Humanas Escola B', 'Aluno Exatas Escola A']
compare_subtitles_em = list(chosens_em)
print(compare_subtitles_em)
compare_columns = [ competencias_dict[key]['codigo'] for key in perfil_em.keys() ]
compare_values_em = list(
    [[profile['perfil_em'][key] for key in perfil_em.keys()] for student, profile in load_students_profiles.items() if student in chosens_em]
)

chosens_enem = ['Aluno Linguagens Escola A', 'Aluno Humanas Escola B', 'Aluno Natureza Escola B']
compare_subtitles_enem = list(chosens_enem)
compare_values_enem = list(
    [[profile['perfil_enem'][key] for key in perfil_em.keys()] for student, profile in load_students_profiles.items() if student in chosens_enem]
)
# Plotar o gráfico de barras




bar_plot(compare_columns, compare_subtitles_em, compare_values_em, True)
bar_plot(compare_columns, compare_subtitles_enem, compare_values_enem, True)


In [ ]:
#Exibir similaridade de perfil_em entre cada estudante de load_students_profiles: Perfil aluno Humanas Escola B e Perfil aluno Linguagens Escola A, Perfil aluno Humanas Escola B e Perfil aluno Natureza Escola A, etc.... 
def calculate_similarity(profile1, profile2, profile=None):
    profile_type1 = 'perfil_em' 
    profile_type2 = 'perfil_enem'
    if profile is not None:
        profile_type1 = profile_type2 = profile
    
    em_array1 = np.array(list(profile1[profile_type1].values()))
    em_array2 = np.array(list(profile2[profile_type2].values()))
    
    similarity_cos = cosine_similarity([em_array1], [em_array2])[0][0]
    theta = math.acos(similarity_cos)
    similarity = 1 - theta / (math.pi / 2.0)
    
    return similarity

# Calcular similaridade entre os perfis
similarities_em = {}
similarities_enem = {}
for key1, profile1 in load_students_profiles.items():
    for key2, profile2 in load_students_profiles.items():
        if key1 != key2:
            similarity_em = calculate_similarity(profile1, profile2, profile='perfil_em')
            similarities_em[(key1, key2)] = similarity_em
            similarity_enem = calculate_similarity(profile1, profile2, profile='perfil_enem')
            similarities_enem[(key1, key2)] = similarity_enem

# Exibir similaridades
print("Similaridades entre os perfis EM:")
for (key1, key2), similarity in similarities_em.items():
    print(f'Similaridade entre {key1} e {key2}: {similarity * 100:.2f}%')

print("Similaridades entre os perfis ENEM:")
for (key1, key2), similarity in similarities_enem.items():
    print(f'Similaridade entre {key1} e {key2}: {similarity * 100:.2f}%')

# Exibir similaridade entre os perfis EM e ENEM de cada estudante

print("Similaridades entre os perfis EM e ENEM:")
for student, profile in load_students_profiles.items():
    similarity_em_enem = calculate_similarity(profile, profile)
    print(f'Similaridade entre o perfil EM e ENEM do {student}: {similarity_em_enem * 100:.2f}%')



